In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

import torch
from transformers import RobertaModel, RobertaTokenizer
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from collections import defaultdict
import string

import time

2025-06-04 20:58:55.483073: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1749070735.861790      13 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1749070735.975895      13 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# testing with a small file
!head -1000 /kaggle/input/tcd-corpus/TCD_All_Micro.txt > /kaggle/working/TCD_All_Micro.txt-small
#corpus_file = "/kaggle/working/TCD_All_Micro.txt-small"

# full corpus
#corpus_file = "/kaggle/input/tcd-corpus/TCD_All_Micro.txt"  

/kaggle/input/tcd-corpus/TCD_All_Micro.txt


In [3]:
!echo "blackboard" > /kaggle/working/common-list.txt
!echo "academic registry" >> /kaggle/working/common-list.txt
!head -2 /kaggle/working/common-list.txt

blackboard
academic registry


In [4]:

# Inputs and outputs
corpus_file = "/kaggle/working/TCD_All_Micro.txt-small"      # ES corpus (one sentence per line)
#corpus_file = "/kaggle/input/tcd-corpus/TCD_All_Micro.txt"      # ES corpus (one sentence per line)
query_file = "/kaggle/working/common-list.txt"       # One query word per line
output_file = "/kaggle/working/synonyms.txt"     # Solr-compatible output

In [5]:
# Initialize RoBERTa
model_name = 'roberta-base'
tokenizer = RobertaTokenizer.from_pretrained(model_name)
model = RobertaModel.from_pretrained(model_name)
model.eval()

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/481 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/499M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


RobertaModel(
  (embeddings): RobertaEmbeddings(
    (word_embeddings): Embedding(50265, 768, padding_idx=1)
    (position_embeddings): Embedding(514, 768, padding_idx=1)
    (token_type_embeddings): Embedding(1, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): RobertaEncoder(
    (layer): ModuleList(
      (0-11): 12 x RobertaLayer(
        (attention): RobertaAttention(
          (self): RobertaSdpaSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): RobertaSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
            (dr

In [6]:
def preprocess_text(text):
    """Lowercase and remove punctuation."""
    text = text.lower().translate(str.maketrans('', '', string.punctuation))
    return text


In [7]:
def get_word_embeddings(corpus_file, max_words=10000):
    """Extract word embeddings line by line."""
    word_counts = defaultdict(int)
    word_embeddings = defaultdict(list)

    with open(corpus_file, 'r', encoding='utf-8') as f:
        for line in f:
            line = preprocess_text(line)
            inputs = tokenizer(line, return_tensors="pt", truncation=True, padding=True)
            with torch.no_grad():
                outputs = model(**inputs)
            embeddings = outputs.last_hidden_state.mean(dim=1).squeeze().numpy()

            for word in line.split():
                if word in tokenizer.get_vocab() and word_counts[word] < 10:
                    word_embeddings[word].append(embeddings)
                    word_counts[word] += 1
                if len(word_embeddings) >= max_words:
                    break

    # Average embeddings per word
    return {word: np.mean(embs, axis=0) for word, embs in word_embeddings.items()}

In [8]:
def find_synonyms(query_words, word_embeddings, top_k=5, similarity_threshold=0.7):
    """Find synonyms for each query word (Solr format)."""
    results = {}
    for word in query_words:
        if word not in word_embeddings:
            continue
        query_embed = word_embeddings[word].reshape(1, -1)
        similarities = {
            w: cosine_similarity(query_embed, e.reshape(1, -1))[0][0]
            for w, e in word_embeddings.items() if w != word
        }
        # Filter by similarity threshold and get top_k
        filtered = [(w, sim) for w, sim in similarities.items() if sim >= similarity_threshold]
        filtered.sort(key=lambda x: x[1], reverse=True)
        synonyms = [w for w, sim in filtered[:top_k]]
        if synonyms:
            results[word] = synonyms
    return results



In [9]:
# Calculate the start time
start = time.time()

In [10]:
if __name__ == "__main__":
    # Input files
    #corpus_file = "corpus.txt"       # Your corpus (one sentence per line)
    #query_file = "queries.txt"       # One query word per line
    #output_file = "synonyms.txt"     # Solr-compatible output

    # Read query words
    with open(query_file, 'r', encoding='utf-8') as f:
        query_words = [preprocess_text(line.strip()) for line in f if line.strip()]

    print("Generating embeddings...")
    word_embeddings = get_word_embeddings(corpus_file)

    print("\nFinding synonyms (Solr format)...")
    synonyms = find_synonyms(query_words, word_embeddings)

    # Write to Solr synonyms.txt format
    with open(output_file, 'w', encoding='utf-8') as f:
        for word, syns in synonyms.items():
            line = f"{word}, {', '.join(syns)}\n"
            f.write(line)
    
    print(f"\nDone! Solr synonyms saved to {output_file}.")

Generating embeddings...

Finding synonyms (Solr format)...

Done! Solr synonyms saved to /kaggle/working/synonyms.txt.


In [11]:
end = time.time()
length = end - start

# Time taken to execute the above (circa 31 min for full corpus, 109 min for n=5000)
print("RoBERTa embeddings took", length/60, "minutes!")

RoBERTa embeddings took 4.250430377324422 minutes!


In [12]:
!wc -l /kaggle/working/synonyms.txt
!head -10 /kaggle/working/synonyms.txt

0 /kaggle/working/synonyms.txt
